In [59]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lipnetbest/tensorflow2/default/1/fin_lipnet_model.h5


In [60]:
import numpy as np
import os
import cv2
from keras.models import Model
from keras.layers import SpatialDropout3D, Input, Conv3D, BatchNormalization, Activation, MaxPooling3D, Bidirectional, LSTM, Dense, TimeDistributed, ZeroPadding3D, Flatten, Dropout, GRU
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, LSTM, Dense, Dropout, Bidirectional, MaxPool3D, Activation, Reshape, SpatialDropout3D, BatchNormalization, TimeDistributed, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
import os
from typing import List
from matplotlib import pyplot as plt
import imageio

In [61]:
! pip install gdown

In [62]:
import gdown

In [63]:
url = 'https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL'
output = 'data.zip'
gdown.download(url, output, quiet=False)
gdown.extractall('data.zip')

Downloading...
From (original): https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL
From (redirected): https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL&confirm=t&uuid=f39a301f-9e1e-4f5e-b507-f9f13b84a8e2
To: /kaggle/working/data.zip
100%|██████████| 423M/423M [00:01<00:00, 218MB/s] 


['data/',
 'data/alignments/',
 'data/alignments/s1/',
 'data/alignments/s1/bbaf2n.align',
 'data/alignments/s1/bbaf3s.align',
 'data/alignments/s1/bbaf4p.align',
 'data/alignments/s1/bbaf5a.align',
 'data/alignments/s1/bbal6n.align',
 'data/alignments/s1/bbal7s.align',
 'data/alignments/s1/bbal8p.align',
 'data/alignments/s1/bbal9a.align',
 'data/alignments/s1/bbas1s.align',
 'data/alignments/s1/bbas2p.align',
 'data/alignments/s1/bbas3a.align',
 'data/alignments/s1/bbaszn.align',
 'data/alignments/s1/bbaz4n.align',
 'data/alignments/s1/bbaz5s.align',
 'data/alignments/s1/bbaz6p.align',
 'data/alignments/s1/bbaz7a.align',
 'data/alignments/s1/bbbf6n.align',
 'data/alignments/s1/bbbf7s.align',
 'data/alignments/s1/bbbf8p.align',
 'data/alignments/s1/bbbf9a.align',
 'data/alignments/s1/bbbm1s.align',
 'data/alignments/s1/bbbm2p.align',
 'data/alignments/s1/bbbm3a.align',
 'data/alignments/s1/bbbmzn.align',
 'data/alignments/s1/bbbs4n.align',
 'data/alignments/s1/bbbs5s.align',
 'data/al

In [64]:
import tensorflow as tf
model = tf.keras.models.load_model("/kaggle/input/lipnetbest/tensorflow2/default/1/fin_lipnet_model.h5", compile=False)

In [65]:
def CTCLoss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [66]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss=CTCLoss)

In [67]:
def load_video(path:str) -> List[float]:

    cap = cv2.VideoCapture(path)
    frames = []
    for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        ret, frame = cap.read()
        frame = tf.image.rgb_to_grayscale(frame)
        frames.append(frame[190:236,80:220,:])
    cap.release()

    mean = tf.math.reduce_mean(frames)
    std = tf.math.reduce_std(tf.cast(frames, tf.float32))
    return tf.cast((frames - mean), tf.float32) / std

In [68]:
vocab = [x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]

In [69]:
char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size ={char_to_num.vocabulary_size()})"
)


The vocabulary is: ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' '] (size =40)


In [70]:
def load_alignments(path:str) -> List[str]:
    file_name = path.split('/')[-1].split('.')[0]
    with open(path, 'r') as f:
        lines = f.readlines()
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil':
            tokens = [*tokens,' ',line[2]]
    return char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]

In [71]:
def load_data(path: str):
    path = bytes.decode(path.numpy())
    file_name = path.split('/')[-1].split('.')[0]
    # File name splitting for windows
    #file_name = path.split('\\')[-1].split('.')[0]
    video_path = os.path.join('data','s1',f'{file_name}.mpg')
    alignment_path = os.path.join('data','alignments','s1',f'{file_name}.align')
    frames = load_video(video_path)
    alignments = load_alignments(alignment_path)

    return frames, alignments

In [72]:
def mappable_function(path:str) ->List[str]:
    result = tf.py_function(load_data, [path], (tf.float32, tf.int64))
    return result

In [73]:
sample = load_data(tf.convert_to_tensor('/kaggle/working/data/s1/sgap2p.mpg'))

In [74]:
print('~'*100, 'REAL TEXT')
[tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in [sample[1]]]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ REAL TEXT


[<tf.Tensor: shape=(), dtype=string, numpy=b'set green at p two please'>]

In [75]:
yhat = model.predict(tf.expand_dims(sample[0], axis=0))

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [76]:
decoded = tf.keras.backend.ctc_decode(yhat, input_length=[75], greedy=True)[0][0].numpy()

In [77]:
print('~'*100, 'PREDICTIONS')
tensor=[tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in decoded]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ PREDICTIONS


In [78]:
# Convert the tensor to a Python string
string_value = tensor[0].numpy().decode("utf-8")

# Print the result
print(string_value)


set green an z fwo please


In [79]:
! pip install ffmpeg-python speechrecognition

In [84]:
import ffmpeg
import speech_recognition as sr
import os

def convert_video_to_wav(video_file, output_audio="extracted_audio.wav"):
    """
    Convert a video file to .wav audio format using ffmpeg.
    """
    try:
        # Convert video to audio using ffmpeg
        ffmpeg.input(video_file).output(output_audio).run(overwrite_output=True)
        print(f"Audio extracted and saved as {output_audio}")
        return output_audio
    except Exception as e:
        print(f"Error converting video to audio: {e}")
        return None

def audio_to_text(audio_file):
    """
    Convert an audio file (wav format) to text using Google Speech Recognition.
    """
    recognizer = sr.Recognizer()

    try:
        # Load the audio file
        with sr.AudioFile(audio_file) as source:
            print("Recognizing audio...")
            audio_data = recognizer.record(source)

            # Convert speech to text using Google's Speech-to-Text
            text = recognizer.recognize_google(audio_data)
            print("Transcribed Text: ", text)
            return text
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand the audio")
        return None
    except sr.RequestError as e:
        print(f"Error with Google Speech Recognition service: {e}")
        return None

def video_to_text(video_file):
    """
    Complete process: Convert a video file to text by extracting audio and
    applying speech-to-text conversion.
    """
    # Step 1: Convert the video to wav format
    audio_file = convert_video_to_wav(video_file)

    if audio_file:
        # Step 2: Convert the extracted audio to text
        text = audio_to_text(audio_file)
        return text
    else:
        print("Audio extraction failed.")
        return None

# Example usage
if __name__ == "__main__":
    video_path = "/kaggle/working/data/s1/prac7s.mpg"  # Replace with the path to your video
    transcribed_text = video_to_text(video_path)

    if transcribed_text:
        # Optionally, save the transcribed text to a file
        with open("transcribed_text.txt", "w") as f:
            f.write(transcribed_text)
        print("Transcription saved to transcribed_text.txt")


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

Audio extracted and saved as extracted_audio.wav
Recognizing audio...
Transcribed Text:  play suede at C7 soon
Transcription saved to transcribed_text.txt


In [85]:
# Mapping of digits to their string representation
num_to_str = {
    '0': 'zero',
    '1': 'one',
    '2': 'two',
    '3': 'three',
    '4': 'four',
    '5': 'five',
    '6': 'six',
    '7': 'seven',
    '8': 'eight',
    '9': 'nine'
}

def convert_string(input_str):
    # Convert the input string to lowercase
    input_str = input_str.lower()
    
    # Replace numbers with their string representation
    result = ''.join([num_to_str[char] if char in num_to_str else char for char in input_str])
    
    return result

In [86]:
import difflib

def sequence_matcher_diff(str1, str2):
    # Create a SequenceMatcher object and compute the similarity ratio
    similarity = difflib.SequenceMatcher(None, str1, str2).ratio()
    return similarity

In [87]:
count=0
transcribed_text=convert_string(transcribed_text)
string_value=convert_string(string_value)

score=(sequence_matcher_diff(transcribed_text, string_value))
if(score>0.5):
    print("Not a Deepfake")
else:
    print("Deepfake")

Deepfake
